In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
steam_games = pd.read_parquet('datasets/steam_games.parquet')
users_items = pd.read_parquet('datasets/users_items.parquet')
users_reviews = pd.read_parquet('datasets/users_reviews.parquet')

In [11]:
muestra_steam_games_modelo = steam_games.head(20000)
muestra_users_reviews_modelo = users_reviews.head(20000)

In [12]:
def recomendacion_juego(game_id:int, top_n=5): #Creo la funcion que toma como parametros la id del juego en game_id y los otros son hiperparametros que defino
    tfidf_vectorizer = TfidfVectorizer()

    #Con esto los datos de la columna genero se transforman en vectores numericos para porder entrenarlos y luego los entreno
    tfidf_matrix = tfidf_vectorizer.fit_transform(muestra_steam_games_modelo['genres'])

    #Aca compara los cosenos de los vectores para encontrar similitudes
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Busca el indice del juego del que ingrese el ID
    idx = None
    idx_list = muestra_steam_games_modelo.index[muestra_steam_games_modelo['item_id'] == game_id].tolist()
    recommended_indices = []  # Inicializa como una lista vacía
    if idx_list:
        idx = idx_list[0]
    # Continúa con el procesamiento
    if idx is not None:
    # No se encontró ninguna coincidencia, maneja esto adecuadamente
        sim_scores = list(enumerate(cosine_sim[idx]))# Optiene la puntuacion de similitud de los cosenos
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # Ordena los juegos por la similitud
        top_n = int(top_n)
        recommended_indices = [i[0] for i in sim_scores[1:top_n+1]]  # Obtiene los indices de los juegos recomendados
    return set(muestra_steam_games_modelo['item_name'].iloc[recommended_indices]) # Devuelve los titulos de los juegos recomendados

In [13]:
def recomendacion_usuario(id:str):
    aux = muestra_users_reviews_modelo[muestra_users_reviews_modelo['user_id'] == id]
    aux.reset_index(drop=True, inplace=True)

    if not aux.empty:  # Verifica si hay datos en 'aux'
        game_id = aux['item_id'].iloc[0]  # Toma el 'item_id' del primer juego del usuario
        recommended_games = recomendacion_juego(game_id=int(game_id))  # Llama a la función
        return recommended_games  # Devuelve las recomendaciones
    else:
        return []  # Retorna una lista vacía si no se encontraron datos para ese usuario

In [ ]:
recomendacion = recomendacion_usuario('evcentric')
recomendacion